# Learning Liar's dice with Q learning

<ins>Author:</ins> Alexandre Forestier--Foray  (https://github.com/Aff54)  
<ins>Date:</ins> 22/01/2026

In [2]:
# ---- Regular Packages ----
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import os
from collections import namedtuple
import pandas as pd

# Reinforcement learning.
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler

In [4]:
# ---- Custom packages ----

from src.reinforcement_learning import ReplayMemory
from src.action_management import get_legal_actions_mask, get_possible_actions
from src.deterministic_agents import agent_random, agent_max_probability, agent_min_probability, Agent
from src.rl_agents import RLAgent
from src.game import GameRL
from src.performance_analysis.analysis_tools import test_agents, get_player_score
from src.performance_analysis.visual_tools import plot_pie_charts
from src.rl_agents import RLAgentOnline, EpsilonScheduler

# 1. Liar's dice game class

<ins>Note:</ins>
The Game class implements the core mechanics of Liar’s Dice. It is responsible for managing player hands, turn order and interactions (calling "liar" or "exact"). It provides an interface for simulating a game.

GameRL subclass extends Game by adding reinforcement learning oriented methods.